In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, pandas as pd, os, pickle
import pandas as pd
import time

# CSV 파일 불러오기
df = pd.read_csv("C:/Users/user/Desktop/label.csv")  # 기존 데이터 파일 (id, label)
df["id"] = df["id"].astype(str).str.replace("^u", "", regex=True)  # "u" 제거
# 🔐 사용자 정보 입력
TWITTER_ID = "아이디"
TWITTER_PW = "비번"

COOKIE_PATH = "twitter_cookies.pkl"

# 🔧 Chrome 설정
options = Options()
#options.add_argument("--headless")  # 창 없이 실행
# options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920x1080")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

def save_cookies(driver, path):
    with open(path, 'wb') as filehandler:
        pickle.dump(driver.get_cookies(), filehandler)

def load_cookies(driver, path):
    with open(path, 'rb') as cookiesfile:
        cookies = pickle.load(cookiesfile)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)

def is_logged_in(driver):
    driver.get("https://twitter.com/home")
    time.sleep(3)
    try:
        driver.find_element(By.XPATH, '//a[@href="/home"]')
        driver.find_element(By.XPATH, '//div[@aria-label="Account menu"]')
        return True
    except:
        return False

def login_and_save_cookies():
    driver.get("https://twitter.com/login")
    time.sleep(3)

    try:
        # ID 입력
        id_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "text"))
        )
        id_input.send_keys(TWITTER_ID)
        id_input.send_keys(Keys.RETURN)
        time.sleep(2)

        # uid_input = WebDriverWait(driver, 10).until(
        #     EC.presence_of_element_locaed((By.NAME, "text"))
        # )
        # uid_input.send_keys(USER_OID)
        # id_input.send_keys(Keys.RETURN)
        # time.sleep(2)
        
        # PW 입력
        pw_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "password"))
        )
        pw_input.send_keys(TWITTER_PW)
        pw_input.send_keys(Keys.RETURN)
        time.sleep(5)
        

        if is_logged_in(driver):
            print("✅ 로그인 성공")
            save_cookies(driver, COOKIE_PATH)
            print("✅ 쿠키 저장 완료")
        else:
            print("❌ 로그인 실패: 로그인 후 프로필을 찾을 수 없음")
            driver.quit()
            exit()

    except Exception as e:
        print("❌ 로그인 실패:", e)
        driver.quit()
        exit()

# 쿠키 파일이 존재하면 불러오고, 없으면 로그인 시도
driver.get("https://twitter.com")
time.sleep(3)

if os.path.exists(COOKIE_PATH):
    try:
        load_cookies(driver, COOKIE_PATH)
        driver.refresh()
        time.sleep(3)

        if not is_logged_in(driver):
            print("⚠️ 쿠키로 로그인 실패, 새로 로그인 시도")
            login_and_save_cookies()
        else:
            print("✅ 쿠키로 로그인 성공")
    except Exception as e:
        print("⚠️ 쿠키 불러오기 실패, 새로 로그인 시도:", e)
        login_and_save_cookies()
else:
    login_and_save_cookies()

# 이제 로그인 세션 유지된 상태에서 크롤링 시작

user_ids = df["id"] # 예시

results = []

for user_id in user_ids:
    try:
        driver.get(f"https://twitter.com/i/user/{user_id}")
        time.sleep(3)

        current_url = driver.current_url
        username = current_url.split("/")[-1]

        # 기본 정보 수집
        try:
            display_name = driver.find_element(By.XPATH, '//div[@data-testid="UserName"]//span[1]').text
        except:
            display_name = ""

        try:
            bio = driver.find_element(By.XPATH, '//div[@data-testid="UserDescription"]').text
        except:
            bio = ""

        try:
            profile_pic = driver.find_element(By.XPATH, '//img[@alt="Image"]').get_attribute("src")
        except:
            profile_pic = ""

        try:
            followers = driver.find_element(By.XPATH, '//a[contains(@href,"/followers")]//span[1]/span').text
        except:
            followers = ""

        try:
            following = driver.find_element(By.XPATH, '//a[contains(@href,"/following")]//span[1]/span').text
        except:
            following = ""

        try:
            posts = driver.find_element(By.XPATH, '//a[contains(@href,"/with_replies")]//span[1]/span').text
        except:
            posts = ""

        nums_in_username = sum(c.isdigit() for c in username)
        nums_in_fullname = sum(c.isdigit() for c in display_name)
        external_url = "http" in bio
        is_private = "This account is private" in driver.page_source

        results.append({
            "user_id": user_id,
            "username": username,
            "display_name": display_name,
            "profile_pic": profile_pic,
            "fullname words": len(display_name.split()),
            "nums/len username": f"{nums_in_username}/{len(username)}",
            "nums/len fullname": f"{nums_in_fullname}/{len(display_name)}",
            "name==username": display_name.lower() == username.lower(),
            "description length": len(bio),
            "external URL": external_url,
            "private": is_private,
            "#posts": posts,
            "#followers": followers,
            "#follows": following,
        })

        print(f"✅ @{username} 크롤링 완료")

    except Exception as e:
        print(f"❌ {user_id} 실패: {e}")

driver.quit()

df = pd.DataFrame(results)
df.to_csv("x_user_data.csv", index=False, encoding='utf-8-sig')
print("📦 저장 완료: x_user_data.csv")

❌ 로그인 실패: Message: 
Stacktrace:
	GetHandleVerifier [0x0087FC03+61635]
	GetHandleVerifier [0x0087FC44+61700]
	(No symbol) [0x006A05D3]
	(No symbol) [0x006E899E]
	(No symbol) [0x006E8D3B]
	(No symbol) [0x00730E12]
	(No symbol) [0x0070D2E4]
	(No symbol) [0x0072E61B]
	(No symbol) [0x0070D096]
	(No symbol) [0x006DC840]
	(No symbol) [0x006DD6A4]
	GetHandleVerifier [0x00B04523+2701795]
	GetHandleVerifier [0x00AFFCA6+2683238]
	GetHandleVerifier [0x00B1A9EE+2793134]
	GetHandleVerifier [0x008968C5+155013]
	GetHandleVerifier [0x0089CFAD+181357]
	GetHandleVerifier [0x00887458+92440]
	GetHandleVerifier [0x00887600+92864]
	GetHandleVerifier [0x00871FF0+5296]
	BaseThreadInitThunk [0x77497BA9+25]
	RtlInitializeExceptionChain [0x77E4C10B+107]
	RtlClearBits [0x77E4C08F+191]

❌ 1217628182611927040 실패: HTTPConnectionPool(host='localhost', port=54769): Max retries exceeded with url: /session/b78a8aa526b763ce24019066bf7b32e5/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0